### **Exercise: Sentiment Analysis and Key Insights Extraction from Ford Car Reviews**

### **Problem Statement:**
You have been provided with a dataset containing Ford car reviews. Your task is to use LangChain and the concepts you’ve learned to perform the following tasks:

1. **Sentiment Analysis**: Analyze the sentiment of each review, categorize it as positive, neutral, or negative, and store the result.
2. **Key Insights Extraction**: Extract key pieces of information from each review, such as the pros and cons mentioned, and the specific features the reviewer liked or disliked (e.g., vehicle performance, comfort, price).

You will build a LangChain-based solution that leverages language models to automatically extract this information and provide a structured summary of the reviews. 

---
### **Steps to Solve:**

#### **Step 1: Load the Dataset**
- The dataset file is named `ford_car_reviews.csv` and is sourced from Kaggle: [Edmunds Consumer Car Ratings and Reviews](https://www.kaggle.com/datasets/ankkur13/edmundsconsumer-car-ratings-and-reviews).
- For this exercise, **limit the data to the first 25 records**. This can be achieved by using `df.head(25)` or `df.iloc[:25]` when loading the data into a DataFrame.

#### **Step 2: Define the Sentiment Analysis Task**
- Use LangChain to create a pipeline to classify the sentiment of each review.
- Define prompts that can guide the model to evaluate the sentiment. For example:
  - "Given the following car review, classify the sentiment as positive, neutral, or negative."

#### **Step 3: Key Insights Extraction**
- Use LangChain to create a pipeline to extract pros, cons, and notable features from each review. Define prompts such as:
  - "What are the pros and cons of the vehicle described in the following review?"
  - "What specific features of the vehicle does the reviewer like or dislike?"

#### **Step 4: Update the DataFrame with New Information**
- Run the pipeline for each review and collect the sentiment and insights.
- Once the analysis and extraction are complete, update the original DataFrame with additional columns to include:
  - Sentiment (positive, neutral, negative)
  - Pros
  - Cons
  - Liked_Features
  - Disliked_Features

---

### **Example Output:**

```json
{
  "Review_Date": "03/07/13",
  "Vehicle_Title": "2006 Ford Mustang Coupe",
  "Review_Text": "With the expected arrival of our 6th child...",
  "Rating": 4.125,
  "Sentiment": "Positive",
  "Pros": "Good driving experience, Large seating capacity, Great options",
  "Cons": "None mentioned",
  "Liked_Features": ["Driving experience", "Seating capacity", "Options available"],
  "Disliked_Features": []
}
```

In [23]:
#pip install ChatGroq
!pip install langchain_groq

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
import pandas as pd 
import numpy as np 
import os, json, re, getpass
from dotenv import load_dotenv
from langchain_groq import ChatGroq


In [25]:
load_dotenv(".env", override=True)
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("GROQ API Key: ")

In [26]:
model_id = "llama3-8b-8192" #llama3-8b-8192, llama-3.1-8b-instant, llama3-groq-8b-8192-tool-use-preview, llama3-groq-70b-8192-tool-use-preview
llm = ChatGroq(model_name=model_id, temperature=0, )

#### PromptTemplate

In [27]:
from langchain_core.prompts import ChatPromptTemplate
system_message_template = """Analyze the sentiment of each review, categorize it as positive, neutral, or negative, and show the result.
                                Secondly, Identify the following details:
                                - Pros: List the positive aspects of the vehicle mentioned in the review.
                                - Cons: List the negative aspects of the vehicle mentioned in the review.
                                - Liked Features: Highlight specific features the reviewer liked (if mentioned).
                                - Disliked Features: Highlight specific features the reviewer disliked (if mentioned).
                                Respond with a JSON object containing these fields: Sentiment, Pros, Cons, Liked_Features, Disliked_Features.

Review_Date : {review_date}
author : {author}
Review_Title : {review_title}
Review_Text : {review_text}
Sentiment : {sentiment}
"""
template = ChatPromptTemplate([
    ("system", system_message_template),
    ("human", "{user_input}")])

In [28]:
df = pd.read_csv("E:/JIO QTR 3/LLM/Lab1/Exercise 1/ford_car_reviews.csv", encoding='utf-8', on_bad_lines='skip', engine='python')
df = df.head(25)

In [29]:
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

parser = JsonOutputParser()   

In [30]:
proof_read_chain = template | llm | JsonOutputParser()

In [31]:
results = []
for _, row in df.iterrows():
    chain_output = proof_read_chain.invoke(dict(
        user_input="Analyze the sentiment of each review, categorize it as positive, neutral, or negative, and show the result in JSON format.",
        author=row['Author_Name'],
        review_date=row['Review_Date'],
        review_title=row['Review_Title'],
        review_text=row['Review'],
        sentiment="" 
    ))
    results.append(chain_output)

print(results)

[{'Sentiment': 'Positive', 'Pros': [], 'Cons': [], 'Liked_Features': [], 'Disliked_Features': []}, {'Sentiment': 'Mixed', 'Pros': ["smokin' hot looking car", 'fun to drive', 'good power', 'great mileage'], 'Cons': ['orneriest transmission', 'difficult to master smooth gear changes', 'harsh ride', 'a lot of road noise'], 'Liked_Features': ['engine sound', 'smooth take-off'], 'Disliked_Features': ['transmission', 'gear changes', 'ride', 'road noise']}, {'Sentiment': 'Positive', 'Pros': ['Great investment', 'Reasonable price'], 'Cons': [], 'Liked_Features': ['Overall ride'], 'Disliked_Features': []}, {'Sentiment': 'Positive', 'Pros': ['The car is a Mustang', 'The car is fast', 'The car has a unique sound'], 'Cons': [], 'Liked_Features': ['V6 engine', 'Air aid cold air injector', 'Throttle body spacer', 'Flowmaster exhaust', 'Blacked out 17 inch factory rims'], 'Disliked_Features': []}, {'Sentiment': 'Positive', 'Pros': ['hugs the road', 'does whatever you ask at a moments notice', 'no mec